# Pandas learning with Birthweight dataset

## Preparing data and libs

In [18]:
import pandas as pd
import numpy as np

np.random.seed(6)   # set rng
pd.options.display.float_format ="{:,.3f}".format # float precision 3 pts
file_path = './Birthweight.csv'

df = pd.read_csv(file_path)   # read data to frame
df.columns = df.columns.map(lambda x: str.lower(x))  # lowercase column names
df.set_index('id', inplace=True)   # id column is index
df.head() 


,length,birthweight,headcirc,gestation,smoker,mage,mnocig,mheight,mppwt,fage,fedyrs,fnocig,fheight,lowbwt,mage35
id,,,,,,,,,,,,,,,
1360,56,4.550,34,44,0,20,0,162,57,23,10,35,179,0,0
1016,53,4.320,36,40,0,19,0,171,62,19,12,0,183,0,0
462,58,4.100,39,41,0,35,0,172,58,31,16,25,185,0,1
1187,53,4.070,38,44,0,20,0,174,68,26,14,25,189,0,0
553,54,3.940,37,42,0,24,0,175,66,30,12,0,184,0,0


## Task 1. pd.Series object, slicing, indexing

In [19]:
sl = df["birthweight"].astype(np.float32).copy()   # Series object
sl.head()

id
1360   4.550
1016   4.320
462    4.100
1187   4.070
553    3.940
Name: birthweight, dtype: float32

In [20]:
print('Direct indexing:')
print(sl.loc[1360:462])   # ids from output above
print('Indirect indexing:')
print(sl.iloc[:3])   # treating like a list

Direct indexing:
id
1360   4.550
1016   4.320
462    4.100
Name: birthweight, dtype: float32
Indirect indexing:
id
1360   4.550
1016   4.320
462    4.100
Name: birthweight, dtype: float32


## Task 2. Add new column to Dataframe. Operations on columns

In [22]:
# add column showing if both parents are smoking
df['bothsmoke'] = (df['mnocig'] > 0) & (df['fnocig'] > 0)   

# ADD LINE
# new_line = pd.DataFrame({'length': [50]})
# df = pd.concat([df, new_line], ignore_index=True)
# df = df.append(new_line, ignore_index=True)  ---deprecated---

# REMOVE LINE
# df.drop([619, 1369])

# REMOVE COLUMN
# df.drop(columns=['length'])
df.tail()



,length,birthweight,headcirc,gestation,smoker,mage,mnocig,mheight,mppwt,fage,fedyrs,fnocig,fheight,lowbwt,mage35,bothsmoke
id,,,,,,,,,,,,,,,,
619,52,3.410,33,39,1,23,25,181,69,23,16,2,181,0,0,True
1369,49,3.180,34,38,1,31,25,162,57,32,16,50,194,0,0,True
1262,53,3.190,34,41,1,27,35,163,51,31,16,25,185,0,0,True
516,47,2.660,33,35,1,20,35,170,57,23,12,50,186,1,0,True
1272,53,2.750,32,40,1,37,50,168,61,31,16,0,173,0,1,False


## Task 3. Set index, describe, change column data type, groupby, aggregate

In [23]:
# setting index
# df.set_index('id', inplace=True)
df.describe()

,length,birthweight,headcirc,gestation,smoker,mage,mnocig,mheight,mppwt,fage,fedyrs,fnocig,fheight,lowbwt,mage35
count,42.000,42.000,42.000,42.000,42.000,42.000,42.000,42.000,42.000,42.000,42.000,42.000,42.000,42.000,42.000
mean,51.333,3.313,34.595,39.190,0.524,25.548,9.429,164.452,57.500,28.905,13.667,17.190,180.500,0.143,0.095
std,2.936,0.604,2.400,2.643,0.505,5.666,12.512,6.504,7.198,6.864,2.160,17.308,6.978,0.354,0.297
min,43.000,1.920,30.000,33.000,0.000,18.000,0.000,149.000,45.000,19.000,10.000,0.000,169.000,0.000,0.000
25%,50.000,2.940,33.000,38.000,0.000,20.250,0.000,161.000,52.250,23.000,12.000,0.000,175.250,0.000,0.000
50%,52.000,3.295,34.000,39.500,1.000,24.000,4.500,164.500,57.000,29.500,14.000,18.500,180.500,0.000,0.000
75%,53.000,3.647,36.000,41.000,1.000,29.000,15.750,169.500,62.000,32.000,16.000,25.000,184.750,0.000,0.000
max,58.000,4.570,39.000,45.000,1.000,41.000,50.000,181.000,78.000,46.000,16.000,50.000,200.000,1.000,1.000


In [24]:
df.dtypes

length           int64
birthweight    float64
headcirc         int64
gestation        int64
smoker           int64
mage             int64
mnocig           int64
mheight          int64
mppwt            int64
fage             int64
fedyrs           int64
fnocig           int64
fheight          int64
lowbwt           int64
mage35           int64
bothsmoke         bool
dtype: object

In [26]:
df = df.astype({'birthweight': np.float16})
df.dtypes['birthweight']

dtype('float16')

In [32]:
# group all smokers and nonsmokers, aggregate birthweigt etc
df.groupby('smoker').agg({'gestation': np.median, 'fnocig': sum, 'birthweight': max})

,gestation,fnocig,birthweight
smoker,,,
0,40.000,194,4.551
1,39.000,528,4.570


In [31]:
# select subarray by determined parameter
df[df['birthweight'] > 4].sort_values('birthweight')

,length,birthweight,headcirc,gestation,smoker,mage,mnocig,mheight,mppwt,fage,fedyrs,fnocig,fheight,lowbwt,mage35,bothsmoke
id,,,,,,,,,,,,,,,,
1187,53,4.070,38,44,0,20,0,174,68,26,14,25,189,0,0,False
462,58,4.102,39,41,0,35,0,172,58,31,16,25,185,0,1,False
1016,53,4.320,36,40,0,19,0,171,62,19,12,0,183,0,0,False
1360,56,4.551,34,44,0,20,0,162,57,23,10,35,179,0,0,False
1764,58,4.570,39,41,1,32,12,173,70,38,14,25,180,0,0,True


## Task 4. Create similar DataFrames, use merge and concat

In [36]:
df1 = pd.DataFrame({
    'name': ['John', 'Bill', 'Valery'],
    'length': [50, 51, 52],
    'birthweight': [3.5, 3.7, 4.0],
    'smoker': [1, 0, 0],
})

df2 = pd.DataFrame({
    'calls': ['John', 'Bill', 'Valery'],
    'gestation': [40, 44, 48],
    'fage': [20, 25, 30],
})

# merge params
# how = outer, left, right (like sql join)
# suffixes = for same column names
df3 = pd.merge(df1, df2, left_on='name', right_on='calls').drop('calls', axis=1)
df3

,name,length,birthweight,smoker,gestation,fage
0,John,50,3.500,1,40,20
1,Bill,51,3.700,0,44,25
2,Valery,52,4.000,0,48,30


In [40]:
df4 = pd.concat([df1, df2], axis=1).drop('calls', axis=1)
df4

,name,length,birthweight,smoker,gestation,fage
0,John,50,3.500,1,40,20
1,Bill,51,3.700,0,44,25
2,Valery,52,4.000,0,48,30
